In [9]:
import pandas as pd
import tkinter as tk
from tkinter import ttk
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side
import win32api,win32con

# 查重类型全局变量
choose = None
# 创建表
wb = Workbook()
ws = wb.active
ws.title = "查重结果"

In [10]:
# 判断字符串中是否同时含有英文与数字
def check_string(s):
    has_letter = False
    has_digit = False
    for char in s:
        if char.isalpha():
            has_letter = True
        elif char.isdigit():
            has_digit = True
        if has_letter and has_digit:
            return True
    return False

# 数据修正
def vehicle_correction(vehicle):
    vehicle_split = vehicle.split(' ')
    if check_string(vehicle_split[-1]) and '-' in vehicle_split[-1]:
        year_split = vehicle_split[-1].split('-')
        length_year = len(year_split[-1])*2 + 1
        vehicle = vehicle[:-length_year] + ' ' + vehicle[-length_year:]
    elif check_string(vehicle_split[-1]) and '-' not in vehicle_split[-1]:
        str_end = vehicle_split[-1][-4:]
        if check_string(str_end):
            vehicle = vehicle[:-2] + ' ' + vehicle[-2:]
        else:
            vehicle = vehicle[:-4] + ' ' + vehicle[-4:]
    else:
        vehicle = vehicle
    return vehicle.replace('  ', ' ').replace(', ', ',').replace('（', '(').replace('）', ')')

# 数据标准化
def vehicle_normalization(vehicle):
    vehicle = vehicle_correction(vehicle)
    if ',' in vehicle:
        vehicle_split = vehicle.split(',')
        if ' ' in vehicle_split[-1]:
            make = vehicle_split[0].split(' ')[0]
            if make in ['Land', 'LAND', 'Alfa', 'ALFA', 'Aston', 'ASTON']:
                make = vehicle_split[0].split(' ')[0] + ' ' + vehicle_split[0].split(' ')[1]
            list_vehicle = [vehicle_split[0]]
            for v in vehicle_split[1:]:
                vehicle_ = make + ' ' + v
                list_vehicle.append(vehicle_)
            vehicle = '\n'.join(list_vehicle)
        else:
            make_model = ' '.join(vehicle_split[0].split(' ')[:-1])
            list_vehicle = [vehicle_split[0]]
            for v in vehicle_split[1:]:
                vehicle_ = make_model + ' ' + v
                list_vehicle.append(vehicle_)
            vehicle = '\n'.join(list_vehicle)
    return vehicle.replace('(', '').replace(')', '').strip()

# 年份判断
def year_(year):
    if len(year) == 2:
        if year in ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09'] or year in [str(i) for i in range(30)]:
            return '20' + year
        else:
            return '19' + year
    else:
        return year

# 年份标准化
def vehicle_handle_1(vehicle):
    if '-' not in vehicle.split(' ')[-1]:
        return year_(vehicle.split(' ')[-1])
    else:
        years = vehicle.split(' ')[-1].split('-')
        year_1 = years[-1]
        year_1 = year_(year_1)
        year_2 = years[0]
        year_2 = year_(year_2)
        year_min = str(min(int(year_1), int(year_2)))
        year_max = str(max(int(year_1), int(year_2)))
        year_result = year_min + '-' + year_max
        return year_result

# 车型标准化
def vehicle_handle_2(vehicle):
    try:
        vehicle_split = vehicle.split(' ')
        make_model = ' '.join(vehicle_split[:-1])
        years = vehicle_handle_1(vehicle)
        return make_model + ' ' + years
    except:
        return ''

In [11]:
def template(choose):
    if choose == 1:
        df_input = pd.read_excel('./模板_1.xlsx', dtype=str)
        df_input['去重字段'] = df_input['Part Number']  + df_input['Year'] + df_input['Make'] + df_input['Model']
        df_input.drop_duplicates(subset=['去重字段'],keep='last',inplace=True)
    else:
        df_input = pd.read_excel('./模板_2.xlsx', dtype=str)
        df_input['Vehicle'] = df_input['Vehicle'].apply(lambda x: x.split('\n'))
        df_input = df_input.explode('Vehicle').fillna('')
        df_input['Vehicle'] = df_input['Vehicle'].apply(lambda x:vehicle_normalization(x))

        df_input['Vehicle'] = df_input['Vehicle'].apply(lambda x: x.split('\n'))
        df_input = df_input.explode('Vehicle').fillna('')
        df_input['Vehicle'] = df_input['Vehicle'].apply(lambda x:vehicle_handle_2(x))

        # 加入车型清洗
        list_vehicle = df_input['Vehicle'].tolist()
        ls_make = []
        ls_model = []
        ls_years = []
        for v in list_vehicle:
            list_split = v.split(' ')
            make = list_split[0]
            year = list_split[-1]
            if make in ['Land', 'Aston', 'Alfa']: # 不影响结果
                make = ' '.join(list_split[:2])
                model = ' '.join(list_split[2:-1])
            else:
                model = ' '.join(list_split[1:-1])
            years = []
            if '-' not in year:
                years.append(year)
            else:
                max_year = int(max(year.split('-')))
                min_year = int(min(year.split('-')))
                for y in range(min_year, max_year+1):
                    years.append(y)
            ls_make.append(make)
            ls_model.append(model)
            ls_years.append(years)
        df_input['Make'] = ls_make
        df_input['Model'] = ls_model
        df_input['Year'] = ls_years
        df_input = df_input.explode('Year').fillna('')
        df_input = df_input[['Part Number', 'Year', 'Make', 'Model']].copy()
        df_input['Year'] = df_input['Year'].apply(lambda x: str(x))
    return df_input

In [12]:
def groupby_number(df, field):
    df = df.groupby('Part Number', as_index=False, sort=False).agg({field:list})
    df[field] = df[field].apply(lambda x: ';'.join(sorted(list(set(x)))))
    df[field] = df[field].apply(lambda x: sorted(list(set(x.split(';')))))
    df[field] = df[field].apply(lambda x: [ele for ele in x if ele!= ''])
    return df

def list_cleand(lst):
    new_list = []
    lst = list(set(lst))
    lst.sort()
    for x, y in zip(lst, lst[1:]):
        if int(y) - int(x) > 1:
            new_list.append(lst[:lst.index(y)])
            lst = lst[lst.index(y):]
    new_list.append(lst)
    return new_list

def handle_year(x):
    if len(x) > 1:
        return str(min(x)) + '-' + str(max(x))
    elif len(x) == 0:
        return ''
    else:
        return x[0]

def Vehicle(df):
    df['Make_Model'] = df['Make'] + ' ' + df['Model']
    df_MM_Y = df.groupby(['Part Number', 'Make_Model'], as_index=False, sort=False).agg({'Year': list})
    df_MM_Y['Year'] = df_MM_Y['Year'].apply(lambda x: list_cleand(x))
    df_MM_Y = df_MM_Y.explode('Year').fillna('')
    df_MM_Y['Year'] = df_MM_Y['Year'].apply(lambda x: handle_year(x))
    df_MM_Y['Vehicle'] = df_MM_Y['Make_Model'] + ' ' + df_MM_Y['Year']
    df_ = df_MM_Y.groupby('Part Number', as_index=False, sort=False).agg({'Vehicle': list})
    df_['Vehicle'] = df_['Vehicle'].apply(lambda x: '\n'.join(x))
    return df_

In [13]:
def Find_complete_repeat(df_table_all_repeat):
    df_all_repeat_number_2 = groupby_number(df_table_all_repeat, '完整重复号码')
    df_repeat_number_explode_2 = df_all_repeat_number_2.explode('完整重复号码').fillna('')
    df_find_number_2 = pd.merge(df_repeat_number_explode_2, df_repeat_number_explode_2, left_on='完整重复号码', right_on='Part Number', how='left').fillna('')
    del df_repeat_number_explode_2
    
    df_find_number_2['Part Number'] = df_find_number_2['Part Number_x']
    df_find_number_2['完整重复号码'] = df_find_number_2['完整重复号码_x'] + ';' + df_find_number_2['完整重复号码_y']
    df_table_all_repeat_2 = df_find_number_2[['Part Number', '完整重复号码']].copy()
    del df_find_number_2
    
    df_table_all_repeat_2 = df_table_all_repeat_2[df_table_all_repeat_2['完整重复号码'] != ';']
    df_table_all_repeat_2 = groupby_number(df_table_all_repeat_2, '完整重复号码')
    df_table_all_repeat_2['完整重复号码'] = df_table_all_repeat_2['完整重复号码'].apply(lambda x: ';'.join(x) if x != None else '')

    del df_all_repeat_number_2, df_table_all_repeat
    return df_table_all_repeat_2
        
def Vehicle_Duplicate_Check(df_input):
    df_input['M_M_Y'] = df_input['Make'] + ' ' + df_input['Model'] + ' ' + df_input['Year']
    df_input['M_M_Y'] = df_input['M_M_Y'].apply(lambda x: str(x).title())
    df_repeat_MMY = df_input.groupby('M_M_Y', as_index=False, sort=False).agg({'Part Number': list})
    df_repeat_MMY['重复号码'] = df_repeat_MMY['Part Number'].apply(lambda x: ';'.join(sorted(list(set(x)), key=x.index)) if len(x) > 1 else '')
    df_repeat_number = pd.merge(df_input, df_repeat_MMY.drop(['Part Number'], axis=1), on='M_M_Y', how='left').fillna('')

    df_all_repeat_number = groupby_number(df_repeat_number, '重复号码')
    df_repeat_number_explode = df_all_repeat_number.explode('重复号码').fillna('')
    df_all_repeat_number['重复号码'] = df_all_repeat_number['重复号码'].apply(lambda x: ';'.join(x) if x != None else '')

    df_find_number = pd.merge(df_repeat_number_explode, df_repeat_number_explode, left_on='重复号码', right_on='Part Number', how='left').fillna('')
    df_find_number['Part Number'] = df_find_number['Part Number_x']
    df_find_number['完整重复号码'] = df_find_number['重复号码_x'] + ';' + df_find_number['重复号码_y']

    df_table_all_repeat = df_find_number[['Part Number', '完整重复号码']]
    df_table_all_repeat = df_table_all_repeat[df_table_all_repeat['完整重复号码'] != ';']
    df_table_all_repeat = groupby_number(df_table_all_repeat, '完整重复号码')
    df_table_all_repeat['完整重复号码'] = df_table_all_repeat['完整重复号码'].apply(lambda x: ';'.join(x) if x != None else '')

    df_table_all_repeat['计数'] = df_table_all_repeat['完整重复号码'].apply(lambda x: x.count(';'))
    total_sum_1 = df_table_all_repeat['计数'].sum()
    df_table_all_repeat = df_table_all_repeat.drop('计数', axis=1)

    while True:
        df_table_all_repeat = Find_complete_repeat(df_table_all_repeat)
        df_table_all_repeat['计数'] = df_table_all_repeat['完整重复号码'].apply(lambda x: x.count(';'))
        total_sum_2 = df_table_all_repeat['计数'].sum()

        if total_sum_1 == total_sum_2:
            df_table_all_repeat = df_table_all_repeat.drop('计数', axis=1)
            break
        else:
            total_sum_1 = total_sum_2
    # for i in range(5):
    #     df_table_all_repeat = Find_complete_repeat(df_table_all_repeat)
    
    df_repeat = pd.merge(df_all_repeat_number, df_table_all_repeat, on='Part Number', how='left').fillna('')
    df_vehicle = Vehicle(df_input)
    df_result_1 = pd.merge(df_vehicle, df_repeat, on='Part Number', how='left').fillna('')
    
    del df_table_all_repeat, df_repeat, df_input, df_vehicle
    
    df_repeat_vehicle = df_repeat_number[df_repeat_number['重复号码'] != '']
    df_repeat_vehicle = df_repeat_vehicle[['重复号码', 'Year', 'Make', 'Model']]
    df_repeat_vehicle.rename(columns={'重复号码':'Part Number'}, inplace=True)
    df_repeat_vehicle_groupby = Vehicle(df_repeat_vehicle)
    df_repeat_vehicle_groupby['重复车型'] = df_repeat_vehicle_groupby['Part Number'] + ':\n' + df_repeat_vehicle_groupby['Vehicle']
    df_repeat_vehicle_groupby['重复车型'] = df_repeat_vehicle_groupby['重复车型'].apply(lambda x: x.replace('\n', '\n    '))
    df_repeat_vehicle_groupby['Part Number'] = df_repeat_vehicle_groupby['Part Number'].apply(lambda x: x.split(';'))

    df_repeat_vehicle_explode = df_repeat_vehicle_groupby.explode('Part Number').fillna('')
    df_merge_vehicle = pd.merge(df_all_repeat_number, df_repeat_vehicle_explode, on='Part Number', how='left').fillna('')
    df_result_2 = df_merge_vehicle.groupby(['Part Number'], as_index=False, sort=False).agg({'重复车型':list})
    df_result_2['重复车型'] = df_result_2['重复车型'].apply(lambda x: '\n'.join(x))

    del df_all_repeat_number, df_merge_vehicle, df_repeat_vehicle_explode, df_repeat_number, df_repeat_vehicle_groupby, df_repeat_vehicle
    
    df_result = pd.merge(df_result_1, df_result_2, on='Part Number', how='left').fillna('')
    df_result.rename(columns={'Vehicle':'车型'}, inplace=True)
    df_result['计数-完整重复号码'] = ''
    df_result['是否存在车型重复'] = ''
    df_result['计数-完整重复号码'] = df_result['完整重复号码'].apply(lambda x: int(x.count(';'))+1 if x != '' else '-')
    df_result['是否存在车型重复'] = df_result['重复号码'].apply(lambda x: '是' if x != '' else '否')
    return df_result

In [14]:
def on_option_click(option): # 选择一二选项后返回序号
    global choose
    choose = option
    root.destroy()
    return choose

def query(option):
    global input_value
    input_value = entry.get()
    count = input_value.count(';')
    if count > 1:
        win32api.MessageBox(0, '请输入两个号码，以英文分号隔开', '提示', win32con.MB_OK)
    else:
        list_query = input_value.split(';')
    
    if option == 3:
        df_input = pd.read_excel('./模板_1.xlsx', dtype=str)
    else:
        df_input = template(2)

    df_input = df_input[df_input['Part Number'].isin(list_query)]
    df_result = Vehicle_Duplicate_Check(df_input)
    try:
        content = df_result['重复车型'].tolist()[0]
    except:
        content = ''
    if content == '':
        content = '无'
    else:
        content = content[content.index(':')+1:].replace('    ', '')
    output_text.delete(1.0, tk.END)
    output_text.insert(tk.END, f"重复车型: {content}")

def Query_Panel(option):
    # 创建查询面板的新窗口
    query_window = tk.Toplevel(root)
    query_window.title("输入号码查询重复车型")
    query_window.geometry(f"{window_width}x{window_height}+{x}+{y}")

    frame = tk.Frame(query_window, bg="#ECEBFE")
    frame.pack(pady=20)

    # 输入区域及标签
    input_label = tk.Label(frame, text="Part Number（用英文分号隔开）", font=("微软雅黑", 10))
    input_label.grid(row=0, column=0, padx=0)

    global entry
    entry = tk.Entry(frame, width=50, font=("微软雅黑", 10), bd=2, relief=tk.SUNKEN)
    entry.grid(row=1, column=0, padx=10, pady=5)

    query_button = tk.Button(frame, text="查询", command=lambda: query(option), font=("微软雅黑", 10, "bold"), bg="#392BEE", fg="white")
    query_button.grid(row=2, column=0, padx=10, pady=5)

    # 输出区域及标签
    global output_text
    output_text = tk.Text(frame, height=10, width=60, font=("微软雅黑", 10))
    output_text.grid(row=3, column=0, padx=10)

def create_button(parent, text, command): #主界面选项样式
    button = tk.Button(parent, text=text, command=command, relief="flat", width=40,
                       bg="#3867F3", fg="#ffffff", font=("微软雅黑", 10, "bold"))
    button.bind("<Enter>", lambda e: button.configure(bg="#392BEE"))
    button.bind("<Leave>", lambda e: button.configure(bg="#3867F3"))
    button.grid(padx=45, pady=15)
    return button

# 主界面
root = tk.Tk()
root.title("查重类型")
root.configure(bg="#ECEBFE")
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()
window_width = 500
window_height = 350
x = (screen_width - window_width) // 2
y = (screen_height - window_height) // 2
root.geometry(f"{window_width}x{window_height}+{x}+{y}")
main_frame = ttk.Frame(root, padding="20", relief="flat")
main_frame.grid(row=0, column=0, padx=20, pady=20, sticky=(tk.W, tk.E, tk.N, tk.S))

# 主界面选项
create_button(main_frame, "1.车型查重（使用模板1）", lambda: on_option_click(1)).grid(row=0, column=0)
create_button(main_frame, "2.车型查重（使用模板2）", lambda: on_option_click(2)).grid(row=1, column=0)
create_button(main_frame, "3.交互界面查询（使用模板1）", lambda: Query_Panel(3)).grid(row=2, column=0)
create_button(main_frame, "4.交互界面查询（使用模板2）", lambda: Query_Panel(4)).grid(row=3, column=0)

root.mainloop()

In [15]:
def result_to_excel(wb, ws, result): # 创建查重结果表格
    for r_idx, row in enumerate(dataframe_to_rows(result, index=False, header=True), 1):
        ws.append(row)
    ws.column_dimensions['A'].width = 15
    ws.column_dimensions['B'].width = 30
    ws.column_dimensions['C'].width = 20
    ws.column_dimensions['D'].width = 20
    ws.column_dimensions['E'].width = 30
    ws.column_dimensions['F'].width = 18
    ws.column_dimensions['G'].width = 18
    ws.freeze_panes = "A2"
    font = Font(name='微软雅黑', size=9)
    header_font = Font(name='微软雅黑', size=9, color="FFFFFF", bold=True)
    fill = PatternFill(start_color="95B3D7", end_color="95B3D7", fill_type="solid")
    alignment = Alignment(horizontal="center", vertical="center", wrap_text=True)
    border = Border(
        left=Side(style='thin'),
        right=Side(style='thin'),
        top=Side(style='thin'),
        bottom=Side(style='thin')
    )
    for row in ws.iter_rows(min_row=2, max_col=ws.max_column, max_row=ws.max_row):
        for cell in row:
            cell.font = font
            cell.alignment = alignment
            cell.border = border
    for cell in ws[1]:
        cell.font = header_font
        cell.fill = fill
        cell.alignment = alignment
        cell.border = border
    ws.row_dimensions[1].height = 20
    for row in range(2, ws.max_row + 1):
        ws.row_dimensions[row].height = 16.8
    
    wb.save('./车型查重结果.xlsx')

In [16]:
if choose == None: # 关闭窗口
    win32api.MessageBox(0, '无进程，结束', '提示', win32con.MB_OK)
elif choose == 1 or choose == 2:
    df_input = template(choose)
    df_result = Vehicle_Duplicate_Check(df_input)
    result_to_excel(wb, ws, df_result)
    win32api.MessageBox(0, f'使用模板_{choose}查重完成，请点击确定结束进程', '提示', win32con.MB_OK)
else:
    pass